# SQL Injection Demo Walkthrough
This notebook walks through how SQL injection works, how it can be exploited, and how to defend against it using secure coding practices.

Author: [@Mira2720](https://github.com/Mira2720)

## What is SQL Injection?
SQL Injection (SQLi) is a code injection technique that might destroy your database. It's one of the most common web hacking techniques.

In [ ]:
# Simulated insecure SQL query
username = "' OR 1=1 --"
password = "any"
query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
print(query)

## Exploit Scenario
This input: `' OR 1=1 --` will make the WHERE clause always true. Result: attacker logs in without valid credentials.

## Secure Query (using parameterized queries)
This is how you **prevent** SQL injection.

In [ ]:
# Parameterized query example
query = "SELECT * FROM users WHERE username = ? AND password = ?"
params = ('admin', 'admin123')
print("Safe query structure:", query, "\nWith values:", params)

## Summary
- ❌ Avoid string formatting in SQL queries
- ✅ Always use parameterized statements
- ✅ Validate and sanitize user input

Try both versions of the app (`run_insecure.py` and `run_secure.py`) to compare the behavior.


## Understanding the Test Files

We created four test files to verify the functionality and security of both app versions.

| Test File                      | Description                              | Target App      | Expected Result        |
|-------------------------------|------------------------------------------|------------------|------------------------|
| `test_insecure_app_failed.py` | Uses SQL injection to bypass login       | Insecure App     | ✅ Injection should succeed |
| `test_insecure_app_pass.py`   | Tests login with correct credentials     | Insecure App     | ✅ Should allow login  |
| `test_secure_app_failed.py`   | Uses SQL injection to bypass login       | Secure App       | ✅ Injection should fail |
| `test_secure_app_pass.py`     | Tests login with correct credentials     | Secure App       | ✅ Should allow login  |

These tests allow for the demonstration and verification the vulnerability and its fix.


# 🟥 Red Team: Attacker View

This section walks through the attacker's perspective using SQL Injections to exploit a vulnerable login form.


In [ ]:
# Target URLs
insecure_url = "http://127.0.0.1:5000/login"
secure_url = "http://127.0.0.1:5001/login"

# SQL Injection payload
payload = {
    "username": "' OR 1=1 --",
    "password": "anything"
}


In [ ]:
import requests

# Attempt login on both apps
insecure_response = requests.post(insecure_url, data=payload)
secure_response = requests.post(secure_url, data=payload)

print("Insecure App Response:")
print(insecure_response.text)
print("\nSecure App Response:")
print(secure_response.text)


# 🟦 Blue Team: Defender View

This section highlights how the secure app defends against SQL injections using proper coding practices and a secure design structure.


## How SQL Injection is Prevented

The secure version of the application uses several techniques to mitigate SQL injection attacks:

- ✅ **Parameterized queries** via placeholders (e.g., `?`) that separate code from data.
- ✅ **Input sanitization** to reject suspicious or malformed data.
- ✅ **Error handling** that avoids leaking database logic to attackers.
- ✅ **Limited permissions** on database accounts (principle of least privilege).

These best practices help ensure user input cannot manipulate SQL query logic.


In [ ]:
# Sample secure login code from the application
import sqlite3

conn = sqlite3.connect("users.db")
cursor = conn.cursor()

username = "user_input"
password = "pass_input"

cursor.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, password))
result = cursor.fetchone()

if result:
    print("✅ Access granted")
else:
    print("❌ Login failed")


### Best Practices Summary

- Never trust user input — always validate and sanitize.
- Use **parameterized queries** or ORM libraries like SQLAlchemy.
- Avoid building SQL strings with concatenation.
- Log login attempts and monitor for patterns like repeated `' OR 1=1 --`.

Always remember, security is a layered process, the code is your front line, make sure to keep it as safe and as clean as possible!
